In [1]:
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt, animation
from PIL import Image
from matplotlib.colors import ListedColormap

In [3]:
class BurningForest():
    # 0 - empty, 1 - occupied, -1 - burning, 2 - burned

    def __init__(self, L, wind=False, windStrength=0):
        self.L = L
        self.forest = np.zeros((L, L))
        
        self.allLabels = np.arange(L*L)
    
    def initiateForest(self, p):
        self.forest = np.zeros((self.L, self.L))
        for i in range(self.L):
            for j in range(self.L):
                if np.random.uniform() < p:
                    self.forest[i, j] = 1
                    
    def initiateFire(self):
        for i in range(self.L):
            state = self.forest[i, 0]
            if state == 1:
                self.forest[i, 0] = -1
                self.burning.append((i, 0))
                
    def get_neighbors(self, pos):
        i = pos[0]
        j = pos[1]
        neighbors = [(i-1, j-1), (i-1, j), (i+1, j), (i, j+1), (i, j-1), (i-1, j+1), (i+1, j-1), (i+1, j+1)]
        cleaned_neighbors = [(x, y) for x, y in neighbors if (x >= 0 and x < self.n and y >= 0 and y < self.m)]
        return cleaned_neighbors
    
    def performBurning(self, numberOfSimulations=100, wind=False):
        ps = np.arange(0.01, 1, 0.01)
        res = np.zeros(len(ps))
        for p in range(len(ps)):
            for i in range(numberOfSimulations):
                self.initiateForest(ps[p])
                if (not wind): 
                    res[p] += self.simulateFire()
                else:
                    res[p] += self.simulateFireWithWind()
            res[p] /= numberOfSimulations
        return res
    
    def simulateFire(self):
        self.initiateFire()
        fireOngoing = True
        while fireOngoing:
            self.burning = list(zip(*np.where(self.forest == -1)))
            if len(self.burning) == 0:
                return 0
            for tree in self.burning:
                neighbors = self.get_neighbors(tree)
                for neighbor in neighbors:
                    i, j = neighbor
                    if self.forest[i, j] == 1:
                        if j >= self.L-1:
                            return 1
                        self.forest[i, j] = -1
                self.forest[tree[0], tree[1]] = 2
        return 0

In [4]:
def animateFire(data, gif_name="anim"):
    fig = plt.figure()
    sns.heatmap(data[0], vmax=.8, square=True, cbar=False, cmap=ListedColormap(['red', 'black', 'green']), yticklabels=False, xticklabels=False)

    def init():
        sns.heatmap(data[0], vmax=.8, square=True, cbar=False, cmap=ListedColormap(['red', 'black', 'green']), yticklabels=False, xticklabels=False)

    def animate(i):
        sns.heatmap(data[i], vmax=.8, square=True, cbar=False, cmap=ListedColormap(['red', 'black', 'green']), yticklabels=False, xticklabels=False)

    anim = animation.FuncAnimation(fig, animate, init_func=init, frames=len(data), repeat = False)
    anim.save(f'images/{gif_name}.gif')

In [5]:
def clean_results(res, L):
    new_res = list(np.zeros(int(len(res)/L-1)))
    for i in range(len(new_res)):
        new_res[i] = res[int((i+1)*L): int((i+2)*L)]
    return new_res